In [1]:
import streamlit as st
import pandas as pd
from google.oauth2 import service_account
from  gspread_pandas import Spread, Client
import ssl
import datetime
import time
import plotly.express as px
import plotly.graph_objects as go


ssl.create_default_https_context = ssl._create_unverified_context

scope = ["https://spreadsheets.google.com/feeds","https://www.googleapis.com/auth/drive"]
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"],
    scopes=scope)

clients = Client(scope=scope,creds=credentials)

spreadsheet_name = "Spotify_database"
spread = Spread(spreadsheet_name,client=clients)

sh = clients.open(spreadsheet_name)
worksheet_list =sh.worksheets()


# Functions
@st.cache_resource()
# Get our worksheet names
def worksheet_names():
    sheet_names = []
    for sheet in worksheet_list:
        sheet_names.append(sheet.title)
    return sheet_names

# Get the sheet as dataframe
def load_the_spreadsheet(spreadsheetname):
    worksheet = sh.worksheet(spreadsheetname)
    df = pd.DataFrame(worksheet.get_all_records())
    return df



In [3]:
historical_data_df = load_the_spreadsheet('historical_data')
classification_df = load_the_spreadsheet('classification')
historical_data_dates_track_types_df = load_the_spreadsheet('historical_dates_types_grouped')
historical_data_dates_df = load_the_spreadsheet('historical_dates')
number_of_artists_yearly = historical_data_df['artistName'].nunique()
number_of_tracks_yearly = historical_data_df['trackName'].nunique()
minutes_per_weekday_total = historical_data_dates_df.groupby(['weekday'],as_index=False).agg(minutes_played=('minutesPlayed','sum'))
minutes_per_weekday_type = historical_data_dates_track_types_df.groupby(['typeObject','weekday'],as_index=False).agg(minutes_played=('listeningTracks','sum'))
print(minutes_per_weekday_total.columns,minutes_per_weekday_type.columns)

Index(['weekday', 'minutes_played'], dtype='object') Index(['typeObject', 'weekday', 'minutes_played'], dtype='object')


In [4]:
fig = go.Figure(data=[go.Bar(
    x=minutes_per_weekday_total['weekday'],
    y=minutes_per_weekday_total['minutes_played'],
    marker_color=colors # marker color can be a single color value or an iterable
    )])
fig.update_layout(title_text='Minutes listened per weekday(Total)')
fig.show()

KeyError: 'minutesPlayed'